In [1]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, BatchNormalization, LeakyReLU, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.backend import clear_session
import datetime
import csv

In [3]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

Num GPUs Available:  1


In [4]:
# Load data
df = pd.read_csv("/content/drive/MyDrive/DHS/Diabetes/Data/India/Data Balance/oversample_smote.csv")

selected_columns = ['hv104','hml18', 'shb70', 'ha53', 'shb13']

# Split the data into features and target
X = df[selected_columns]
y = df['final_diabetes']

# Convert X and y to numpy arrays if they are not already
X = np.array(X)
y = np.array(y)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for the CNN model
X = X.reshape((X.shape[0], X.shape[1], 1))

In [16]:
# Define the robust CNN-LSTM hybrid model
def cnn_lstm(learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=256, kernel_size=3, padding="same", input_shape=(5, 1)))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    """
    model.add(Conv1D(filters=128, kernel_size=3, padding="same"))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))

    model.add(Conv1D(filters=64, kernel_size=3, padding="same"))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.3))

    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.3))
    """
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [17]:
# Evaluate the model
def evaluate_model(model, X_val, y_val):
    y_val_pred = model.predict(X_val)
    y_val_pred_binary = (y_val_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_val, y_val_pred_binary)
    precision = precision_score(y_val, y_val_pred_binary)
    recall = recall_score(y_val, y_val_pred_binary)
    f1 = f1_score(y_val, y_val_pred_binary)
    roc_auc = roc_auc_score(y_val, y_val_pred)
    cm = confusion_matrix(y_val, y_val_pred_binary)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    kappa = cohen_kappa_score(y_val, y_val_pred_binary)
    mcc = matthews_corrcoef(y_val, y_val_pred_binary)

    metrics = [accuracy, precision, recall, f1, roc_auc, specificity, kappa, mcc, cm]
    return metrics

In [18]:
from sklearn.model_selection import StratifiedKFold

class FederatedData:
    def __init__(self, X, y, num_clients, test_size=0.2):
        self.X = X
        self.y = y
        self.num_clients = num_clients
        self.test_size = test_size
        self.partitions = []
        self.create_partitions()

    def create_partitions(self):
        skf = StratifiedKFold(n_splits=self.num_clients, shuffle=True, random_state=42)
        for train_index, test_index in skf.split(self.X, self.y):
            X_train, X_test = self.X[train_index], self.X[test_index]
            y_train, y_test = self.y[train_index], self.y[test_index]
            self.partitions.append((X_train, y_train, X_test, y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")
        partition_X_train, partition_y_train, partition_X_test, partition_y_test = self.partitions[client_idx]
        X_train, X_val, y_train, y_val = train_test_split(partition_X_train, partition_y_train, test_size=0.2, stratify=partition_y_train, random_state=42)
        return X_train, X_val, y_train, y_val, partition_X_test, partition_y_test

# Initialize federated data
federated_data = FederatedData(X, y, num_clients=3)

Train shape: (949976, 5, 1), Test shape: (474988, 5, 1)
Train shape: (949976, 5, 1), Test shape: (474988, 5, 1)
Train shape: (949976, 5, 1), Test shape: (474988, 5, 1)


In [20]:
import os
import datetime
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import csv

# Custom CSV Logger
class CustomCSVLogger(tf.keras.callbacks.Callback):
    def __init__(self, filename, separator=',', append=False):
        self.sep = separator
        self.filename = filename
        self.append = append
        self.file = None
        self.writer = None
        self.keys = None
        super(CustomCSVLogger, self).__init__()

    def on_train_begin(self, logs=None):
        if not self.append and os.path.isfile(self.filename):
            os.remove(self.filename)
        mode = 'a' if self.append else 'w'
        self.file = open(self.filename, mode, newline='')
        self.writer = csv.writer(self.file, delimiter=self.sep)

    def on_epoch_end(self, epoch, logs=None):
        if self.keys is None:
            self.keys = sorted(logs.keys())
            self.writer.writerow(self.keys)
        row = [logs.get(key) for key in self.keys]
        self.writer.writerow(row)

    def on_train_end(self, logs=None):
        self.file.close()


# Initialize global model
input_shape = X.shape[1:]
global_model = cnn_lstm()
num_clients = 3
local_epochs = 1

# CSV file to store performance metrics
csv_filename = "/content/drive/MyDrive/DHS/Diabetes/Data/India/Deep Learning/Fedarated/5 Features/5_features_federated_learning_metrics.csv"
header = ['Local Epoch', 'Client', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC', 'Specificity', 'Kappa', 'MCC', 'Confusion Matrix']
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

# Federated training process
for local_epoch in range(1, local_epochs + 1):
    print(f"Local Epoch {local_epoch}:")
    client_models = []

    # Create a directory for the current local epoch
    epoch_dir = f"/content/drive/MyDrive/DHS/Diabetes/Data/India/Deep Learning/Fedarated/5 Features/epoch_{local_epoch}"
    os.makedirs(epoch_dir, exist_ok=True)

    for client in range(num_clients):
        X_train, X_val, y_train, y_val, X_test, y_test = federated_data.get_training_and_validation_data(client)
        client_model = cnn_lstm()
        client_model.set_weights(global_model.get_weights())  # Initialize with global weights

        with tf.device('/GPU:0'):
            log_dir = os.path.join(epoch_dir, "logs/profile/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, profile_batch=0)
            csv_logger = CustomCSVLogger(os.path.join(epoch_dir, f"training_client_{client+1}_epoch_{local_epoch}.csv"))

            history = client_model.fit(
                X_train, y_train,
                epochs=50,  # Train for 10 epochs (as a placeholder, adjust as needed)
                batch_size=512,
                validation_data=(X_val, y_val),
                verbose=1,
                callbacks=[csv_logger]
            )

        # Evaluate the client model
        client_metrics = evaluate_model(client_model, X_val, y_val)
        print(f"Client {client + 1} - Metrics after Local Epoch {local_epoch}: {client_metrics}")

        # Save client model performance to CSV
        with open(csv_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([local_epoch, client + 1] + client_metrics[:-1] + [np.array2string(client_metrics[-1], separator=',')])

        client_models.append(client_model)

    # Aggregating weights from client models to update global model
    global_weights = global_model.get_weights()
    new_weights = [client_model.get_weights() for client_model in client_models]

    averaged_weights = [np.mean(np.array([client_weight[layer] for client_weight in new_weights]), axis=0) for layer in range(len(global_weights))]
    global_model.set_weights(averaged_weights)

# Save the global model
global_model.save("/content/drive/MyDrive/DHS/Diabetes/Data/India/Deep Learning/Fedarated/5 Features/5_features_global_model.keras")

Local Epoch 1:
Epoch 1/50
1485/1485 [==============================] - 13s 7ms/step - loss: 0.0590 - accuracy: 0.9747 - val_loss: 0.0531 - val_accuracy: 0.9769
Epoch 2/50
1485/1485 [==============================] - 10s 7ms/step - loss: 0.0534 - accuracy: 0.9766 - val_loss: 0.0540 - val_accuracy: 0.9759
Epoch 3/50
1485/1485 [==============================] - 9s 6ms/step - loss: 0.0523 - accuracy: 0.9768 - val_loss: 0.0503 - val_accuracy: 0.9772
Epoch 4/50
1485/1485 [==============================] - 10s 7ms/step - loss: 0.0514 - accuracy: 0.9770 - val_loss: 0.0512 - val_accuracy: 0.9770
Epoch 5/50
1485/1485 [==============================] - 10s 7ms/step - loss: 0.0514 - accuracy: 0.9771 - val_loss: 0.0493 - val_accuracy: 0.9777
Epoch 6/50
1485/1485 [==============================] - 10s 7ms/step - loss: 0.0513 - accuracy: 0.9770 - val_loss: 0.0489 - val_accuracy: 0.9775
Epoch 7/50
1485/1485 [==============================] - 9s 6ms/step - loss: 0.0507 - accuracy: 0.9771 - val_loss: 0.

KeyboardInterrupt: 